In [14]:
###########################################################
### LOAD LINKEDIN DATA INTO LINKEDIN STAGING TABLE CODE ###
### Version - Final                                     ###
### WRITTEN BY DANA CAINES - PROD DATE = 12/12/23       ###
###########################################################

import pandas as pd
import json
import mysql.connector
from mysql.connector import Error

# Function to check and return alumniID: if alumni does not exist, insert and keep alumniID for adding to row in stage table; else, if alumni exists, get alumniID and add to row in stage table
def get_or_create_alumni_id(firstname, lastname, cursor):
    # Check if alumni exists
    cursor.execute("SELECT alumniID FROM alumni WHERE firstname = %s AND lastname = %s", (firstname, lastname))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        # Insert new alumni record
        cursor.execute("INSERT INTO alumni (firstname, lastname) VALUES (%s, %s)", (firstname, lastname))
    return cursor.lastrowid

#Function to insert record into stage table: if the record exists, then insert; else update the record
def update_insert_row(cursor, conn, data_tuple):
    # Check if the record already exists
    
    firstname = data_tuple[2]
    lastname = data_tuple[3]
    
    cursor.execute("""
        SELECT EXISTS(
            SELECT 1 FROM stg_in_linkedin 
            WHERE  firstname = %s AND lastname = %s
            AND    alumniID is NOT NULL
        )
    """, ('firstname', 'lastname'))
    record_exists = cursor.fetchone()[0]

    if record_exists:
        # Update the existing record
        update_stmt = """
            UPDATE stg_in_linkedin
            SET alumniID = %s,
                dept = %s, 
                url = %s, 
                firstname = %s, 
                lastname = %s, 
                fullname = %s, 
                email_first = %s, 
                email_second = %s, 
                phone = %s, 
                current_title = %s, 
                current_company = %s, 
                company_phone = %s, 
                city = %s, 
                state = %s, 
                country = %s,
                sm_school = %s,
                degree1 = %s,
                field_of_study_1 = %s, 
                school_1 = %s, 
                start_year_1 = %s, 
                end_year_1 = %s, 
                years_school_1 = %s, 
                degree2 = %s, 
                field_of_study_2 = %s, 
                school_2 = %s, 
                start_year_2 = %s, 
                end_year_2 = %s, 
                years_school_2 = %s, 
                degree3 = %s, 
                field_of_study_3 = %s, 
                school_3 = %s, 
                start_year_3 = %s, 
                end_year_3 = %s, 
                years_school_3 = %s, 
                field_of_study_4 = %s, 
                start_year_4 = %s, 
                end_year_4 = %s, 
                years_school_4 = %s, 
                experience = %s, 
                employers = %s, 
                extra_info = %s
            WHERE firstname = %s AND lastname = %s AND alumniID IS NOT NULL
        """

        update_data_tuple = data_tuple[1:-1] + (firstname, lastname)
        # print("Update Data Tuple Length:", len(update_data_tuple))
        
        cursor.execute(update_stmt, update_data_tuple)
    else:
        # Insert new record
        insert_stmt = """
            INSERT INTO stg_in_linkedin (
                alumniID,
                dept, 
                url, 
                firstname, 
                lastname, 
                fullname, 
                email_first, 
                email_second, 
                phone, 
                current_title, 
                current_company, 
                company_phone, 
                city, 
                state, 
                country, 
                sm_school,
                degree1, 
                field_of_study_1, 
                school_1, 
                start_year_1, 
                end_year_1, 
                years_school_1, 
                degree2, 
                field_of_study_2, 
                school_2, 
                start_year_2, 
                end_year_2, 
                years_school_2, 
                degree3, 
                field_of_study_3, 
                school_3, 
                start_year_3, 
                end_year_3, 
                years_school_3, 
                field_of_study_4, 
                start_year_4, 
                end_year_4, 
                years_school_4, 
                experience, 
                employers, 
                extra_info
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
       
    
    insert_data_tuple = data_tuple
    # print("Update Data Tuple Length:", len(insert_data_tuple))
        
    
    cursor.execute(insert_stmt, insert_data_tuple)
        

    # Assuming you're passing the connection object into this function,
    # so we'll commit after each operation.
    conn.commit()
    #cursor.connection.commit()



# Function to read csv and insert into database stage table
def insert_csv_to_stage(file_path, host_name, user_name, user_password, db_name):
    
    try:
        # Read the csv file
        df = pd.read_csv(file_path)
        #print(df.columns)
        
        df = df.where(pd.notnull(df), 'NULL')

        # Connect to the database
        conn = mysql.connector.connect(host=host_name, user=user_name, passwd=user_password, database=db_name)
        print('Connected to db')
        cursor = conn.cursor()

        # Iterate over the DataFrame and insert/update records
        for index, row in df.iterrows():
            try:
                alumni_id = get_or_create_alumni_id(row['first_name'], row['last_name'], cursor)
                data_tuple = (alumni_id,) + tuple(row[col] for col in df.columns)
                update_insert_row(cursor, conn, data_tuple)
                conn.commit()
            except Exception as e:
                print(f"Error at dataframe index {index}: {e}")
                #print("Row content:", row.to_dict())
                error_data_tuple = (str(e),) + (alumni_id,) + tuple(row[col] for col in df.columns)
                insert_error_log(cursor, conn, error_data_tuple, 'insert_csv_to_stage')
                conn.rollback()
        
        print("Records were successfully processed")

    except Error as e:
        print("Error while connecting to MySQL", e)
        #conn.rollback()
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection is closed")
            
def insert_error_log(cursor, conn, error_data_tuple, source):
    try:
        insert_stmt = """INSERT INTO error_log (
            error_message, source,
            alumni_id, dept, url, firstname, lastname, fullname,
            email_first, email_second, phone, current_title, current_company,
            company_phone, city, state, country, sm_school,
            degree1, field_of_study_1, school_1, start_year_1, end_year_1, years_school_1,
            degree2, field_of_study_2, school_2, start_year_2, end_year_2, years_school_2,
            degree3, field_of_study_3, school_3, start_year_3, end_year_3, years_school_3,
            field_of_study_4, start_year_4, end_year_4, years_school_4,
            experience, employers, extra_info
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,  %s, %s)
        """
        
        error_tuple = (str(error_data_tuple[0]), source) + error_data_tuple[1:]
        cursor.execute(insert_stmt, error_tuple, )
        conn.commit()
        
    except Exception as e:
        print(f"Error logging failed: {e}")
        

# Identify credentials and working directory
csv_file_path = '/Users/dcain/Library/CloudStorage/OneDrive-Personal/Saint Marys Course/OPS 659 Practicum/Alumni DB Python Files/Staged_Files/clean_linkedin_12Dec23.csv'
# MySQL connection details
host='localhost'
user='root'
passwd='rusercaines'
database='smc_alumni_test_2'
# host = '34.27.128.119'
# user = 'ahmed',
# passwd = 'abcdefgh',
# database = 'smc_alumni'

# Call the function to read CSV and insert into MySQL
insert_csv_to_stage(csv_file_path, host, user, passwd, database)


Connected to db
Error at dataframe index 118: 1062 (23000): Duplicate entry 'Amber-Coatney' for key 'stg_in_linkedin.firstname'
Error at dataframe index 160: 1062 (23000): Duplicate entry 'Jacqueline-Guerrero' for key 'stg_in_linkedin.firstname'
Error at dataframe index 407: 1062 (23000): Duplicate entry 'Mariana-Vazquez' for key 'stg_in_linkedin.firstname'
Error at dataframe index 412: 1062 (23000): Duplicate entry 'Brandon-Dighero' for key 'stg_in_linkedin.firstname'
Error at dataframe index 487: 1062 (23000): Duplicate entry 'Joe-Ottati' for key 'stg_in_linkedin.firstname'
Error at dataframe index 998: 1062 (23000): Duplicate entry 'Hilary-Calhoun' for key 'stg_in_linkedin.firstname'
Error at dataframe index 1043: 1062 (23000): Duplicate entry 'Paul-Kelly' for key 'stg_in_linkedin.firstname'
Error at dataframe index 1132: 1062 (23000): Duplicate entry 'Amy-Gibbs' for key 'stg_in_linkedin.firstname'
Error at dataframe index 1171: 1062 (23000): Duplicate entry 'Shayan-Hamid' for key '

In [25]:
###########################################################
### PROCESS FROM LINKEDIN STAGING INTO CORE DB TABLES   ###
### Version - Final                                     ###
### WRITTEN BY DANA CAINES - PROD DATE = 12/12/23       ###
###########################################################

import mysql.connector
from mysql.connector import Error

def create_db_connection():
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user='root',
            passwd='rusercaines',
            database='smc_alumni_test_2'
            # host='34.27.128.119',
            # user='ahmed',
            # passwd='abcdefgh',
            # database='smc_alumni'
        )
        return conn
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
        return None


def insert_error_log(cursor, conn, error_data_tuple, source_function):
    insert_stmt = """INSERT INTO error_log (
        source, error_message, alumni_id, ...
    ) VALUES (%s, %s, %s, ...)
    """
    error_tuple = (source_function,) + error_data_tuple
    cursor.execute(insert_stmt, error_tuple)
    conn.commit()
    
    
def process_contact_information():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            
            # Processing logic for contact_information
            select_query = """
                SELECT alumniID, email_first, email_second, phone, company_phone 
                FROM stg_in_linkedin
                #LIMIT 1
            """
            cursor.execute(select_query)
            records = cursor.fetchall()
            
            for record in records:
                alumni_id, email_primary, email_secondary, phone, work_phone = record

                # Check if alumniID exists in contact_information
                cursor.execute("""
                    SELECT EXISTS(
                        SELECT 1 FROM contact_information
                        WHERE alumniID = %s
                    )
                """, (alumni_id,))
                record_exists = cursor.fetchone()[0]
                
                if record_exists:
                    # Update existing record
                    update_query = """
                        UPDATE contact_information
                        SET email_primary = COALESCE(NULLIF(%s, ''), email_primary),
                            email_secondary = COALESCE(NULLIF(%s, ''), email_secondary),
                            phone = COALESCE(NULLIF(%s, ''), phone),
                            company_phone = COALESCE(NULLIF(%s, ''), work_phone)
                        WHERE alumniID = %s
                    """
                    cursor.execute(update_query, (email_primary, email_secondary, phone, work_phone, alumni_id))
                else:
                    # Insert new record
                    insert_query = """
                        INSERT INTO contact_information (
                            alumniID, email_primary, email_secondary, phone, company_phone
                        ) VALUES (%s, %s, %s, %s, %s)
                    """
                    cursor.execute(insert_query, (alumni_id, email_primary, email_secondary, phone, work_phone))

                conn.commit()
          
            cursor.close()
        except Error as e:
            print("Error while processing contact_information:", e)
            conn.rollback()
        finally:
            if conn.is_connected():
                conn.close()

def process_employment():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            
            # Select data from stage table
            select_query = """
                SELECT alumniID, current_title, current_company, city, state, country
                FROM stg_in_linkedin
            """
            cursor.execute(select_query)
            records = cursor.fetchall()
            
            for record in records:
                alumni_id, job_title, employer, city, state, country = record

                # Check if a record with this combination exists in employment
                cursor.execute("""
                    SELECT EXISTS(
                        SELECT 1 FROM employment
                        WHERE alumniID = %s AND job_title = %s AND employer = %s AND job_city = %s
                    )
                """, (alumni_id, job_title, employer, city))
                record_exists = cursor.fetchone()[0]
                
                if record_exists:
                    # Update existing record
                    update_query = """
                        UPDATE employment
                        SET job_title = COALESCE(NULLIF(%s, ''), job_title),
                            employer = COALESCE(NULLIF(%s, ''), employer),
                            job_city = COALESCE(NULLIF(%s, ''), city),
                            job_state = COALESCE(NULLIF(%s, ''), state),
                            job_country = COALESCE(NULLIF(%s, ''), country)
                        WHERE alumniID = %s AND job_title = %s AND employer = %s AND city = %s
                    """
                    cursor.execute(update_query, (job_title, employer, city, state, country, alumni_id, job_title, employer, city))
                else:
                    # Insert new record
                    insert_query = """
                        INSERT INTO employment (
                            alumniID, job_title, employer, job_city, job_state, job_country
                        ) VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    cursor.execute(insert_query, (alumni_id, job_title, employer, city, state, country))

                conn.commit()

        except Error as e:
            print("Error while processing employment:", e)
            error_data_tuple = (str(e),) + record
            insert_error_log(cursor, conn, error_data_tuple, "process_employment")
            conn.rollback()
        finally:
            if conn.is_connected():
                conn.close()

def process_education():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            record = None
            
            # Select data from stage table
            select_query = """
                SELECT alumniID, degree2, field_of_study_2, school_2, start_year_2, end_year_2, years_school_2,
                       degree3, field_of_study_3, school_3, start_year_3, end_year_3, years_school_3,
                       degree4, field_of_study_4, school_4, start_year_4, end_year_4, years_school_4
                FROM stg_in_linkedin
                #LIMIT 2
            """
            cursor.execute(select_query)
            records = cursor.fetchall()
            #print('Processing education ...')
            for record in records:
                alumni_id = record[0]
                
                # Process each degree set
                for i in range(1, 19, 6):
                    degree = record[i]
                    #print(degree)
                    field_of_study = record[i+1]
                    school = record[i+2]
                    start_year = record[i+3]
                    end_year = record[i+4]
                    years_attended = record[i+5]

                    # Check if a record with this combination exists in education
                    cursor.execute("""
                        SELECT EXISTS(
                            SELECT 1 FROM education
                            WHERE alumniID = %s AND degree = %s AND field_of_study = %s AND school = %s
                        )
                    """, (alumni_id, degree, field_of_study, school))
                    record_exists = cursor.fetchone()[0]
                    
                    if record_exists and degree and field_of_study and school:
                        # Update existing record
                        update_query = """
                            UPDATE education
                            SET start_year = COALESCE(NULLIF(%s, ''), start_year),
                                end_year = COALESCE(NULLIF(%s, ''), end_year),
                                years_attended = COALESCE(NULLIF(%s, ''), years_attended)
                            WHERE alumniID = %s AND degree = %s AND field_of_study = %s AND school = %s
                            
                        """
                        cursor.execute(update_query, (start_year, end_year, years_attended, alumni_id, degree, field_of_study, school))
                    elif degree and field_of_study and school:
                        # Insert new record
                        insert_query = """
                            INSERT INTO education (
                                alumniID, degree, field_of_study, school, start_year, end_year, years_attended
                            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                        """
                        cursor.execute(insert_query, (alumni_id, degree, field_of_study, school, start_year, end_year, years_attended))

                    conn.commit()

        except Error as e:
            print("Error while processing education:", e)
            if record:
                error_data_tuple = (str(e),) + record
            else:
                error_data_tuple = (str(e),)
            insert_error_log(cursor, conn, error_data_tuple, "process_education")
            conn.rollback()
        finally:
            if conn is not None and conn.is_connected():
                conn.close()


# Call the functions to process data
#process_contact_information()
#process_employment()
#process_education()
process_mailing_information()
#process_links



In [23]:
def process_mailing_information():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()

            # SQL query to process mailing information
            insert_query = """
            INSERT INTO mailing_information (alumniID, address1, address2, city, state, zip_code, country)
            SELECT mc.alumniID, mc.address1, mc.address2, mc.city, mc.state, mc.zip_code, mc.country
            FROM stg_in_mailchimp mc
            WHERE (mc.address1 IS NOT NULL OR mc.address2 IS NOT NULL OR mc.city IS NOT NULL OR mc.state IS NOT NULL OR mc.zip_code IS NOT NULL OR mc.country IS NOT NULL);
            """
            cursor.execute(insert_query)
            conn.commit()

        except Error as e:
            print("Error while processing mailing information:", e)
            # Here, you might want to log the error to error_log table
            conn.rollback()
        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()

In [ ]:
import mysql.connector
from mysql.connector import Error

def create_db_connection():
    try:
        conn = mysql.connector.connect(
            host='34.27.128.119',
            user='ahmed',
            passwd='abcdefgh',
            database='smc_alumni'
        )
        return conn
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
        return None

def insert_error_log(cursor, conn, error_data_tuple, source_function):
    # insert_stmt = """INSERT INTO error_log (
    #     source, error_message, alumni_id, ...
    # ) VALUES (%s, %s, %s, ...)
    # """
    
    insert_stmt = """INSERT INTO error_log (
            error_message, source,
            alumni_id, dept, url, firstname, lastname, fullname,
            email_first, email_second, phone, current_title, current_company,
            company_phone, city, state, country,
            degree1, field_of_study_1, school_1, start_year_1, end_year_1, years_school_1,
            degree2, field_of_study_2, school_2, start_year_2, end_year_2, years_school_2,
            degree3, field_of_study_3, school_3, start_year_3, end_year_3, years_school_3,
            field_of_study_4, start_year_4, end_year_4, years_school_4,
            experience, employers, extra_info
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,  %s, %s, %s)
        """
    error_tuple = (source_function,) + error_data_tuple
    cursor.execute(insert_stmt, error_tuple)
    conn.commit()

def process_contact_information():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            # Processing logic for contact_information
            # ...
        except Exception as e:
            print(f"Error in process_contact_information: {e}")
            error_data_tuple = (str(e), ...)  # Gather necessary error data
            insert_error_log(cursor, conn, error_data_tuple, 'process_contact_information')
            conn.rollback()
        finally:
            cursor.close()
            conn.close()

def process_employment():
    conn = create_db_connection()
    if conn is not None:
        try:
            cursor = conn.cursor()
            # Processing logic for employment
            # ...
        except Exception as e:
            print(f"Error in process_employment: {e}")
            error_data_tuple = (str(e), ...)  # Gather necessary error data
            insert_error_log(cursor, conn, error_data_tuple, 'process_employment')
            conn.rollback()
        finally:
            cursor.close()
            conn.close()

# ... Additional functions for other tables like process_education, etc.

# Call the functions to process data
process_contact_information()
process_employment()
# ... Call other processing functions as needed
